<a href="https://colab.research.google.com/github/Rishabhc711/Federated_in_Healthcare/blob/main/FinalDia7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import pandas as pd
import numpy as np
import sklearn
import keras
import tensorflow as tf

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))

Python: 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0]
Pandas: 1.1.5
Numpy: 1.19.5
Sklearn: 0.22.2.post1
Keras: 2.5.0


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Data processing, metrics and modeling
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix,  roc_curve, precision_recall_curve, accuracy_score, roc_auc_score
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from yellowbrick.classifier import DiscriminationThreshold

# Stats
import scipy.stats as ss
from scipy import interp
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# Time
from contextlib import contextmanager
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore') 

In [5]:
# Always good to set a seed for reproducibility
SEED = 0
np.random.seed(SEED)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/FInal/diabetes/Cleaned_final/7_cleaned - 7.csv')
df.head()

,Glucose,Age,Sex,BMI,Systolic BP,Diastolic BP,Cholesterol,Outcome,Diabetes,Gender
0,77.0,19.0,0,22.5,118.0,70.0,193.0,0.0,No diabetes,female
1,79.0,19.0,0,26.4,108.0,58.0,146.0,0.0,No diabetes,female
2,75.0,20.0,0,29.3,110.0,72.0,217.0,0.0,No diabetes,female
3,97.0,20.0,0,19.6,122.0,64.0,226.0,0.0,No diabetes,female
4,91.0,20.0,0,20.2,122.0,86.0,164.0,0.0,No diabetes,female


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Glucose       390 non-null    float64
 1   Age           390 non-null    float64
 2   Sex           999 non-null    object 
 3   BMI           390 non-null    float64
 4   Systolic BP   390 non-null    float64
 5   Diastolic BP  390 non-null    float64
 6   Cholesterol   390 non-null    float64
 7   Outcome       390 non-null    float64
 8   Diabetes      390 non-null    object 
 9   Gender        390 non-null    object 
dtypes: float64(7), object(3)
memory usage: 78.2+ KB


In [10]:
df[389:]
#df.drop(15251,axis=0,inplace=True) 

,Glucose,Age,Sex,BMI,Systolic BP,Diastolic BP,Cholesterol,Outcome,Diabetes,Gender
389,94.0,92.0,0,39.7,160.0,82.0,165.0,0.0,No diabetes,female
390,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
391,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
994,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for i in range(390,999):
  df.drop(i,axis=0,inplace=True) 
 


df['Systolic BP']=pd.to_numeric(df['Systolic BP'],downcast="float")
df['Diastolic BP']=pd.to_numeric(df['Diastolic BP'],downcast="float")
df['Outcome']=pd.to_numeric(df['Outcome'],downcast="float")




In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 0 to 389
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Glucose       390 non-null    float64
 1   Age           390 non-null    float64
 2   Sex           390 non-null    object 
 3   BMI           390 non-null    float64
 4   Systolic BP   390 non-null    float32
 5   Diastolic BP  390 non-null    float32
 6   Cholesterol   390 non-null    float64
 7   Outcome       390 non-null    float32
 8   Diabetes      390 non-null    object 
 9   Gender        390 non-null    object 
dtypes: float32(3), float64(4), object(3)
memory usage: 28.9+ KB


In [13]:
df.describe()

,Glucose,Age,BMI,Systolic BP,Diastolic BP,Cholesterol,Outcome
count,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000,390.000000
mean,107.338462,46.774359,28.775641,137.133331,83.289742,207.230769,0.153846
std,53.798188,16.435911,6.600915,22.859537,13.498198,44.666005,0.361265
min,48.000000,19.000000,15.200000,90.000000,48.000000,78.000000,0.000000
25%,81.000000,34.000000,24.100000,122.000000,75.000000,179.000000,0.000000
50%,90.000000,44.500000,27.800000,136.000000,82.000000,203.000000,0.000000
75%,107.750000,60.000000,32.275000,148.000000,90.000000,229.000000,0.000000
max,385.000000,92.000000,55.800000,250.000000,124.000000,443.000000,1.000000


In [14]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(390, 10)


In [15]:
# Distribution for federated learning cycles
fed_cycle_count=1
last=np.int(dataset.shape[0]*(0.8+(0.2*(fed_cycle_count-1))))

In [16]:
# split into input (X) and an output (Y)
X = dataset[:last,0:6]
Y = dataset[:last, 6]#.astype(int)

In [17]:
X=np.asarray(X).astype('float64')
Y=np.asarray(Y).astype('float64')

In [18]:
print(X.shape)
print(Y.shape)

(312, 6)
(312,)


In [19]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [20]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5
count,3.120000e+02,3.120000e+02,3.120000e+02,3.120000e+02,3.120000e+02,3.120000e+02
mean,4.839434e-17,9.323027e-16,1.579933e-16,3.238150e-16,-6.917543e-16,-4.088610e-16
std,1.001606e+00,1.001606e+00,1.001606e+00,1.001606e+00,1.001606e+00,1.001606e+00
min,-1.111526e+00,-1.830622e+00,-8.121419e-01,-1.999669e+00,-2.137144e+00,-2.640747e+00
25%,-4.702836e-01,-8.191331e-01,-8.121419e-01,-6.940303e-01,-6.667184e-01,-6.281122e-01
50%,-3.238580e-01,-6.051643e-02,-8.121419e-01,-1.680969e-01,-1.030554e-01,-1.063180e-01
75%,-2.090868e-02,7.823910e-01,1.231312e+00,5.637965e-01,4.115934e-01,6.391024e-01
max,5.694736e+00,1.793880e+00,1.231312e+00,3.973169e+00,4.724841e+00,3.024447e+00


In [21]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
model= tf.keras.models.load_model('/content/drive/MyDrive/saved_model/my_model1')

In [ ]:
model.load_weights("/content/drive/MyDrive/weights1.h5")

ValueError: ignored

In [27]:
# Start defining the model
def create_model():
    model = Sequential()
    model.add(Dense(6, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(12, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, input_dim = 12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 0)

In [ ]:
# define the grid search parameters
batch_size = [16, 32, 64,128]
epochs = [10,15,20,50]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=SEED), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] batch_size=16, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ batch_size=16, epochs=10, score=0.000, total=   1.6s
[CV] batch_size=16, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] ............ batch_size=16, epochs=10, score=0.000, total=   1.2s
[CV] batch_size=16, epochs=10 ........................................


...

[CV] ............ batch_size=16, epochs=15, score=0.016, total=   1.5s
[CV] batch_size=16, epochs=15 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.5s remaining:    0.0s


[CV] ............ batch_size=16, epochs=15, score=0.000, total=   1.4s
[CV] batch_size=16, epochs=20 ........................................
[CV] ............ batch_size=16, epochs=20, score=0.000, total=   1.9s
[CV] batch_size=16, epochs=20 ........................................
[CV] ............ batch_size=16, epochs=20, score=0.000, total=   1.5s
[CV] batch_size=16, epochs=20 ........................................
[CV] ............ batch_size=16, epochs=20, score=0.000, total=   1.5s
[CV] batch_size=16, epochs=20 ........................................
[CV] ............ batch_size=16, epochs=20, score=0.016, total=   1.6s
[CV] batch_size=16, epochs=20 ........................................
[CV] ............ batch_size=16, epochs=20, score=0.000, total=   1.5s
[CV] batch_size=16, epochs=50 ........................................
[CV] ............ batch_size=16, epochs=50, score=0.000, total=   3.4s
[CV] batch_size=16, epochs=50 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.0min finished


Best: 0.0032258063554763796, using {'batch_size': 16, 'epochs': 10}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 16, 'epochs': 10}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 16, 'epochs': 15}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 16, 'epochs': 20}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 16, 'epochs': 50}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 32, 'epochs': 10}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 32, 'epochs': 15}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 32, 'epochs': 20}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 32, 'epochs': 50}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 64, 'epochs': 10}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 64, 'epochs': 15}
0.0032258063554763796 (0.006451612710952759) with: {'batch_size': 64, 'epochs': 20}
0.003225

In [ ]:

best_batch_size=16
best_epochs=10

In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout


# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(6, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(12, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(6, input_dim = 12, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = best_epochs, batch_size = best_batch_size, verbose = 0)

# define the grid search parameters
learn_rate = [0.000001,0.00001,0.0001, 0.001]
dropout_rate = [0.0, 0.2, 0.4,0.6]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] dropout_rate=0.0, learn_rate=1e-06 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. dropout_rate=0.0, learn_rate=1e-06, score=0.000, total=   1.4s
[CV] dropout_rate=0.0, learn_rate=1e-06 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=1e-06, score=0.000, total=   1.4s
[CV] dropout_rate=0.0, learn_rate=1e-06 ..............................


...

[CV] .. dropout_rate=0.0, learn_rate=1e-05, score=0.016, total=   1.7s
[CV] dropout_rate=0.0, learn_rate=1e-05 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   14.3s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=1e-05, score=0.000, total=   1.7s
[CV] dropout_rate=0.0, learn_rate=0.0001 .............................
[CV] . dropout_rate=0.0, learn_rate=0.0001, score=0.000, total=   1.3s
[CV] dropout_rate=0.0, learn_rate=0.0001 .............................
[CV] . dropout_rate=0.0, learn_rate=0.0001, score=0.000, total=   1.6s
[CV] dropout_rate=0.0, learn_rate=0.0001 .............................
[CV] . dropout_rate=0.0, learn_rate=0.0001, score=0.000, total=   1.7s
[CV] dropout_rate=0.0, learn_rate=0.0001 .............................
[CV] . dropout_rate=0.0, learn_rate=0.0001, score=0.016, total=   1.7s
[CV] dropout_rate=0.0, learn_rate=0.0001 .............................
[CV] . dropout_rate=0.0, learn_rate=0.0001, score=0.000, total=   1.7s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.000, total=   1.3s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.2min finished


Best: 0.006451612710952759, using {'dropout_rate': 0.6, 'learn_rate': 1e-06}
0.0 (0.0) with: {'dropout_rate': 0.0, 'learn_rate': 1e-06}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.0, 'learn_rate': 1e-05}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.0, 'learn_rate': 0.0001}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.2, 'learn_rate': 1e-06}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.2, 'learn_rate': 1e-05}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.2, 'learn_rate': 0.0001}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.4, 'learn_rate': 1e-06}
0.0032258063554763796 (0.006451612710952759) with: {'dropout_rate': 0.4, 'learn_rate': 1e-05}
0.00322580635547

In [ ]:
best_dropout_rate = 0.0
best_learn_rate = 0.000001

In [ ]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(6, input_dim = 6, kernel_initializer= init, activation= activation))
    model.add(Dense(12, input_dim = 6, kernel_initializer= init, activation= activation))
    model.add(Dense(6, input_dim = 12, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    adam = Adam(lr = best_learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = best_epochs, batch_size = best_batch_size, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... activation=softmax, init=uniform, score=0.000, total=   1.6s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.000, total=   1.5s
[CV] activation=softmax, init=uniform ................................


...

[CV] ..... activation=softmax, init=normal, score=0.000, total=   1.6s
[CV] activation=softmax, init=normal .................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.7s remaining:    0.0s


[CV] ..... activation=softmax, init=normal, score=0.000, total=   1.5s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.000, total=   1.5s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.000, total=   1.9s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.000, total=   1.3s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.016, total=   1.3s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.000, total=   1.5s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.000, total=   1.2s
[CV] activation=relu, init=uniform ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.4min finished


Best: 0.0032258063554763796, using {'activation': 'softmax', 'init': 'uniform'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'softmax', 'init': 'uniform'}
0.0 (0.0) with: {'activation': 'softmax', 'init': 'normal'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'softmax', 'init': 'zero'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'relu', 'init': 'uniform'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'relu', 'init': 'normal'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'relu', 'init': 'zero'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'tanh', 'init': 'uniform'}
0.0 (0.0) with: {'activation': 'tanh', 'init': 'normal'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'tanh', 'init': 'zero'}
0.0032258063554763796 (0.006451612710952759) with: {'activation': 'linear', 'init': 'uniform'}
0.0032258063554763796 (0.006451612710952759) with: {'activati

In [ ]:
best_activation = 'softmax'
best_init = 'uniform'

In [ ]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages


# Start defining the model
def create_model(neuron1, neuron2, neuron3):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 6, kernel_initializer= best_init, activation= best_activation))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= best_init, activation= best_activation))
    model.add(Dense(neuron3, input_dim = neuron2, kernel_initializer= best_init, activation= best_activation))
    model.add(Dense(1, activation='sigmoid'))

    # compile the model
    adam = Adam(lr = best_learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = best_epochs, batch_size = best_batch_size, verbose = 0)

# define the grid search parameters
neuron1 = [6, 12, 24,36]
neuron2 = [24, 36,48, 60]
neuron3 = [6, 12,24, 36]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2, neuron3 = neuron3)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] neuron1=6, neuron2=24, neuron3=6 ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... neuron1=6, neuron2=24, neuron3=6, score=0.000, total=   1.6s
[CV] neuron1=6, neuron2=24, neuron3=6 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] .... neuron1=6, neuron2=24, neuron3=6, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=6 ................................


...

[CV] ... neuron1=6, neuron2=24, neuron3=12, score=0.000, total=   1.2s
[CV] neuron1=6, neuron2=24, neuron3=12 ...............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   12.8s remaining:    0.0s


[CV] ... neuron1=6, neuron2=24, neuron3=12, score=0.000, total=   1.6s
[CV] neuron1=6, neuron2=24, neuron3=24 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=24, score=0.000, total=   1.6s
[CV] neuron1=6, neuron2=24, neuron3=24 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=24, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=24 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=24, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=24 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=24, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=24 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=24, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=36 ...............................
[CV] ... neuron1=6, neuron2=24, neuron3=36, score=0.000, total=   1.5s
[CV] neuron1=6, neuron2=24, neuron3=36 ...............................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:  8.1min finished


Best: 0.006451612710952759, using {'neuron1': 6, 'neuron2': 24, 'neuron3': 36}
0.0 (0.0) with: {'neuron1': 6, 'neuron2': 24, 'neuron3': 6}
0.0 (0.0) with: {'neuron1': 6, 'neuron2': 24, 'neuron3': 12}
0.0 (0.0) with: {'neuron1': 6, 'neuron2': 24, 'neuron3': 24}
0.006451612710952759 (0.007901579579944178) with: {'neuron1': 6, 'neuron2': 24, 'neuron3': 36}
0.006451612710952759 (0.007901579579944178) with: {'neuron1': 6, 'neuron2': 36, 'neuron3': 6}
0.0032258063554763796 (0.006451612710952759) with: {'neuron1': 6, 'neuron2': 36, 'neuron3': 12}
0.0032258063554763796 (0.006451612710952759) with: {'neuron1': 6, 'neuron2': 36, 'neuron3': 24}
0.006451612710952759 (0.007901579579944178) with: {'neuron1': 6, 'neuron2': 36, 'neuron3': 36}
0.0032258063554763796 (0.006451612710952759) with: {'neuron1': 6, 'neuron2': 48, 'neuron3': 6}
0.0032258063554763796 (0.006451612710952759) with: {'neuron1': 6, 'neuron2': 48, 'neuron3': 12}
0.0032258063554763796 (0.006451612710952759) with: {'neuron1': 6, 'neuro

In [54]:
best_neuron1 = 6
best_neuron2 = 24
best_neuron3 = 36

In [77]:
best_epochs=10
best_batch_size=16

best_dropout_rate = 0.0
best_learn_rate = 0.000001

best_activation = 'softmax'
best_init = 'uniform'

best_neuron1 = 6
best_neuron2 = 24
best_neuron3 = 36


In [96]:
#best model
model = Sequential()
model.add(Dense(best_neuron1, input_dim = 6, kernel_initializer= best_init, activation= best_activation))
model.add(Dense(best_neuron2, input_dim = best_neuron1, kernel_initializer= best_init, activation= best_activation))
model.add(Dense(best_neuron3, input_dim = best_neuron2, kernel_initializer= best_init, activation= best_activation))
model.add(Dense(1, activation='sigmoid'))

In [97]:
from sklearn.model_selection import train_test_split

In [104]:
#SEED=6
# compile the model
adam = Adam(learning_rate =0.0000009 )# best_learn_rate)
model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
ckpt_model = 'pima-weights_best_t.hdf5'
checkpoint = ModelCheckpoint(ckpt_model, 
                            monitor='val_accuracy', 
                            verbose=1,
                            save_best_only=True,
                            mode='max')
callbacks_list = [checkpoint]

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,Y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

history = model.fit(X_train,
                    y_train,
                    validation_data=(X_valid, y_valid),
                    epochs=best_epochs,
                    batch_size=best_batch_size,
                    callbacks=callbacks_list,
                    verbose=1
                    )

(249, 6)
(249,)
(31, 6)
(31,)
(32, 6)
(32,)
Epoch 1/10
16/16 [==============================] - 1s 24ms/step - loss: 0.0650 - accuracy: 0.0000e+00 - val_loss: 0.0548 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to pima-weights_best_t.hdf5
Epoch 2/10
16/16 [==============================] - 0s 5ms/step - loss: 0.0517 - accuracy: 0.0000e+00 - val_loss: 0.0478 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/10
16/16 [==============================] - 0s 5ms/step - loss: 0.0435 - accuracy: 0.0000e+00 - val_loss: 0.0408 - val_accuracy: 0.0000e+00

Epoch 00003: val_accuracy did not improve from 0.00000
Epoch 4/10
16/16 [==============================] - 0s 5ms/step - loss: 0.0440 - accuracy: 0.0000e+00 - val_loss: 0.0338 - val_accuracy: 0.0000e+00

Epoch 00004: val_accuracy did not improve from 0.00000
Epoch 5/10
16/16 [==============================] - 0s 5ms/step - loss: 0.0424 - accuracy: 0.000

In [105]:
)model.evaluate(X_test,y_test)

SyntaxError: ignored

In [106]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_65 (Dense)             (None, 24)                168       
_________________________________________________________________
dense_66 (Dense)             (None, 36)                900       
_________________________________________________________________
dense_67 (Dense)             (None, 1)                 37        
Total params: 1,147
Trainable params: 1,147
Non-trainable params: 0
_________________________________________________________________


In [108]:
model.save_weights("/content/drive/MyDrive/weights_diabetes7.h5")